**Downloading dataset**

In [1]:
!wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz

--2020-03-28 02:29:53--  http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131740031 (126M) [application/x-tar]
Saving to: ‘101_ObjectCategories.tar.gz’

101_ObjectCategorie 100%[===================>] 125.64M  8.95MB/s    in 16s     

2020-03-28 02:30:15 (7.77 MB/s) - ‘101_ObjectCategories.tar.gz’ saved [131740031/131740031]



**Unzipping dataset file**

In [0]:
!tar -xf 101_ObjectCategories.tar.gz

In [0]:
!rm -rf '101_ObjectCategories/BACKGROUND_Google'

In [4]:
%tensorflow_version 1.x
from keras.preprocessing.image import ImageDataGenerator

TensorFlow 1.x selected.


Using TensorFlow backend.


**In each folder, split 30 images to training set and remaining to test set**

In [0]:
import os
import math

os.mkdir("caltech_test") # stores test data

for cat in os.listdir("101_ObjectCategories/"):
  # moves x portion of images per category into test images
  os.mkdir("caltech_test/"+cat) # new category folder
  imgs = os.listdir("101_ObjectCategories/"+cat) # all image filenames
  test_imgs = imgs[30:len(imgs)]
  for t_img in test_imgs: # move test portion
    os.rename("101_ObjectCategories/"+cat+"/"+t_img, "caltech_test/"+cat+"/"+t_img)

In [0]:
def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)

**Preparing data for feature extraction**

In [0]:
# import cv2
# training_images = []
# validation_images = []
# for root, dirs, files in os.walk('101_ObjectCategories/'):
#     for file in files:
#         # print(file)
#         # print(root + '/' + file)
#         image = cv2.imread(root + '/' + file)
#         dim = (256, 256)
#         # resize image
#         image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
#         training_images.append(image)
  

# for root, dirs, files in os.walk('caltech_test/'):
#     for file in files:
#         image = cv2.imread(root + '/' + file)
#         dim = (256, 256)
#         # resize image
#         image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
#         validation_images.append(image)




from keras.applications.resnet50 import preprocess_input 

train_gen_autoencoder = ImageDataGenerator(rescale=1./255) #rotation_range = 30, zoom_range = 0.20, 
# fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
# width_shift_range = 0.1, height_shift_range = 0.1)
train_flow = train_gen_autoencoder.flow_from_directory("101_ObjectCategories/", target_size=(256, 256), batch_size=64,class_mode=None)
valid_flow = train_gen_autoencoder.flow_from_directory("caltech_test/", target_size=(256, 256), batch_size=64,class_mode=None)

**Importing necessary library**

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

**Autoencoder model**

In [0]:
model = models.Sequential()

#encoder
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), padding='same') )
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), padding='same') )
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), padding='same') )
# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.MaxPooling2D((2, 2), padding='same') )
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2), padding='same') )
# model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same') )
# model.add(layers.MaxPooling2D((2, 2), padding='same') )
# model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same') )
# model.add(layers.MaxPooling2D((2, 2), padding='same') )

#decoder
# model.add(layers.Conv2D(8, (3, 3), activation='relu', padding='same') )
# model.add(layers.UpSampling2D((2, 2)) )
# model.add (layers.Conv2D(8, (3, 3), activation='relu', padding='same') )
# model.add(layers.UpSampling2D((2, 2)) )
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D((2, 2)) )
# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.UpSampling2D((2, 2)) )
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.UpSampling2D((2, 2)) )
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.UpSampling2D((2, 2)) )
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.UpSampling2D((2, 2)) )

model.add(layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same') )

model.compile(optimizer='adadelta', loss='binary_crossentropy')

**Training autoencoder**

In [0]:
# model.fit(training_images, training_images,
#                 epochs=5,
#                 batch_size=128,
#                 shuffle=True,
#                 validation_data=(validation_images, validation_images))
model.fit_generator(fixed_generator(train_flow), epochs=5,steps_per_epoch=3030//128, validation_steps=5647//128,validation_data = fixed_generator(valid_flow) )

**data augmentation and genearting data for trainig with keras flow from directory**

In [11]:
from keras.applications.resnet50 import preprocess_input 

train_gen = ImageDataGenerator(validation_split=0.2, preprocessing_function=preprocess_input) #rotation_range = 30, zoom_range = 0.20, 
# fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
# width_shift_range = 0.1, height_shift_range = 0.1)
train_flow = train_gen.flow_from_directory("101_ObjectCategories/", target_size=(256, 256), batch_size=32, subset="training")
valid_flow = train_gen.flow_from_directory("101_ObjectCategories/", target_size=(256, 256), batch_size=32, subset="validation")

Found 2424 images belonging to 101 classes.
Found 606 images belonging to 101 classes.


In [12]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_flow = test_gen.flow_from_directory("caltech_test", target_size=(256, 256), batch_size=32)

Found 5647 images belonging to 101 classes.


**Preparing model with the help of resnet 50 pretrained model on imagenet**

In [13]:
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from keras.models import Model,Sequential


res = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3)) 
# model = Sequential()

# for layer in res.layers[:-1]:
#     model.add(layer)

for layer in res.layers: # freezing the layers
  layer.trainable = False

# model.add(GlobalAveragePooling2D())
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(101, activation='softmax'))


x = res.output # get the output from the loaded model
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(101, activation='softmax')(x)

model = Model(res.input, x) 

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy']) # compile the model - we're training using the Adam Optimizer and Categorical Cross Entropy as the loss function

model.summary() 

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1

**Training model** 

In [14]:
import time
start = time.time()
history = model.fit_generator(train_flow, epochs=15, validation_data=valid_flow)
end = time.time() 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
76/76 [==============================] - 18s 235ms/step - loss: 3.0834 - acc: 0.3390 - val_loss: 0.7020 - val_acc: 0.8168
Epoch 2/15
76/76 [==============================] - 8s 107ms/step - loss: 0.8523 - acc: 0.7813 - val_loss: 0.4672 - val_acc: 0.8630
Epoch 3/15
76/76 [==============================] - 8s 107ms/step - loss: 0.4927 - acc: 0.8628 - val_loss: 0.3995 - val_acc: 0.8878
Epoch 4/15
76/76 [==============================] - 8s 107ms/step - loss: 0.3508 - acc: 0.9171 - val_loss: 0.3998 - val_acc: 0.8762
Epoch 5/15
76/76 [==============================] - 8s 107ms/step - loss: 0.2523 - acc: 0.9372 - val_loss: 0.3713 - val_acc: 0.8960
Epoch 6/15
76/76 [==============================] - 8s 107ms/step - loss: 0.2036 - acc: 0.9475 - val_loss: 0.3439 - val_acc: 0.9010
Epoch 7/15
76/76 [==============================] - 8s 107ms/step - loss: 0.1438 - acc: 0.9668 - val_loss: 0.34

In [0]:
print("Total Training time: ",(end-start))

In [0]:
import keras
from matplotlib import pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig("performance_caltech101.png")

**Evaluating model**

In [15]:
score = model.evaluate(test_flow)

print('The model achieved a accuracy of %.2f%%.' % (score[1]*100))

177/177 [==============================] - 16s 91ms/step
The model achieved a accuracy of 91.55%.
